In [ ]:
!pip install streamlit
!pip install pyngrok
!pip install scikit-learn openpyxl pandas
!pip install streamlit-option-menu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 55.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 81.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 829.3/829.3 kB 9.7 MB/s eta 0:00:00


In [ ]:
%%writefile app.py
import streamlit as st
import pandas as pd
import numpy as np
import pydeck as pdk
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import hashlib
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.impute import SimpleImputer
from sklearn.metrics.pairwise import pairwise_kernels
from sklearn.cluster import SpectralClustering
from streamlit_option_menu import option_menu
from sklearn.cluster import SpectralClustering
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler



# Set konfigurasi halaman Streamlit
st.set_page_config(page_title="Aplikasi Pengaruh Prakerja dan Klasterisasi Prakerja terhadap Tingkat Pengangguran Jawa Barat", layout="wide")

# Judul aplikasi
st.markdown(
    "<h1 style='text-align: center; color: navy;'>Aplikasi Pengaruh Prakerja dan Klasterisasi Prakerja terhadap Tingkat Pengangguran Jawa Barat</h1>",
    unsafe_allow_html=True
)

# Sidebar menu dengan option_menu
with st.sidebar:
    selected = option_menu(
        menu_title=None,
        options=["Upload Data", "Proses Analisis","Analisis Klaster", "Rata-Rata Klaster","Analisis Pengaruh Prakerja terhadap Pengangguran","Input Data Manual"],
        icons=["cloud-upload", "gear", "bar-chart", "calculator","graph-up-arrow","pencil-square"],
        default_index=0,
        styles={
            "container": {"padding": "5px", "background-color": "#f0f2f6"},
            "icon": {"color": "#2c3e50", "font-size": "18px"},
            "nav-link": {
                "font-size": "16px",
                "text-align": "left",
                "margin": "5px",
                "border-radius": "10px",
                "padding": "10px 20px",
                "background-color": "#e4e6eb",
            },
            "nav-link-selected": {
                "background-color": "#0d6efd",
                "color": "white",
                "font-weight": "bold",
            },
        }
    )

# Menu Upload Data
if selected == "Upload Data":
    st.header("📂 Upload Data")
    st.info("Silakan upload file untuk mulai proses.")

    # Uploader untuk file CSV atau Excel
    uploaded_file = st.file_uploader("Upload file data", type=["xlsx"], key="upload_data")

    if uploaded_file is not None:
        try:
            # Baca file sesuai ekstensi
            if uploaded_file.name.endswith('.csv'):
                df = pd.read_csv(uploaded_file, dtype=str)
            elif uploaded_file.name.endswith('.xlsx'):
                df = pd.read_excel(uploaded_file, engine='openpyxl', dtype=str)
            else:
                st.error("Format file tidak dikenali.")
                st.stop()
            # Bersihkan nama kolom dari spasi
            df.columns = df.columns.str.strip()

            # Simpan ke session
            st.session_state['df'] = df
            st.session_state['uploaded_file'] = uploaded_file

            st.success("File berhasil diupload!")

            # Input jumlah baris yang ingin ditampilkan
            jumlah_baris = st.number_input(
                "Tampilkan berapa data?",
                min_value=1,
                max_value=len(df),
                value=min(100, len(df)),
                step=1
            )

            # Tampilkan data
            st.write(f"Jumlah data: {len(df)} ")
            st.dataframe(df.head(jumlah_baris))

        except Exception as e:
            st.error(f"Gagal memproses file: {e}")

    elif 'df' in st.session_state:
        df = st.session_state['df']
        st.success("Menampilkan data dari sesi sebelumnya")

        jumlah_baris = st.number_input(
            "Tampilkan berapa baris?",
            min_value=1,
            max_value=len(df),
            value=min(100, len(df)),
            step=1
        )

        st.write(f"Jumlah data: {len(df)} baris")
        st.dataframe(df.head(jumlah_baris))

# Menu Proses Analisis
elif selected == "Proses Analisis":
    st.title("🔍 Proses Analisis Data")

    if 'df' not in st.session_state:
        st.warning("Silakan upload data terlebih dahulu di menu 'Upload Data'.")
        st.stop()

    df = st.session_state['df'].copy()

    # Ganti tanda '-' jadi NaN
    df.replace("-", np.nan, inplace=True)

    # Hapus kolom yang terlalu banyak null
    df.dropna(axis=1, thresh=0.1 * len(df), inplace=True)

    # Imputasi kolom jika ada
    imputer = SimpleImputer(strategy="mean")
    for col in ['SD ke bawah', 'Diploma I/II/III/Akademi/Universitas']:
        if col in df.columns:
            df[col] = imputer.fit_transform(df[[col]])

    # Encode Wilayah
    if "Wilayah" in df.columns:
        df["Wilayah_Original"] = df["Wilayah"]
        df["Wilayah_Encoded"] = LabelEncoder().fit_transform(df["Wilayah"])

    # Konversi kolom numerik
    numeric_columns = [
        "Jumlah angkatan Kerja", "Jumlah pengangguran terbuka", "TPT", "laki-laki", "perempuan",
        "SD ke bawah", "SMP", "SMA", "SMK", "Diploma I/II/III/Akademi/Universitas",
        "sk penetapan", "Aktif", "TPAK", "Tahun"
    ]
    for col in numeric_columns:
        if col in df.columns:
            df[col] = df[col].astype(str).str.replace(',', '', regex=True).replace('-', np.nan)
            df[col] = pd.to_numeric(df[col], errors='coerce')

    # Pilih fitur untuk clustering
    features_for_cluster = [
        "TPT", "Jumlah pengangguran terbuka", "Aktif", "TPAK", "Jumlah angkatan Kerja",
        "laki-laki", "perempuan", "SD ke bawah", "SMP", "SMA", "SMK", "Diploma I/II/III/Akademi/Universitas"
    ]
    X = df[features_for_cluster].dropna()

    # Normalisasi
    scaler = MinMaxScaler()
    X_normalized = scaler.fit_transform(X)

    # Tampilkan tabel hasil normalisasi
    st.subheader("Tabel Normalisasi (Min-Max)")
    df_normalized = pd.DataFrame(X_normalized, columns=features_for_cluster)
    st.dataframe(df_normalized)
    st.markdown("""
    Normalisasi digunakan untuk menyamakan skala data antar fitur agar tidak ada fitur yang mendominasi.
    Di sini digunakan metode **Min-Max Scaling** untuk mengubah data ke rentang 0 hingga 1.
    """)

    # Affinity Matrix
    st.subheader("Affinity Matrix (RBF Kernel)")
    gamma = 1.0
    affinity_matrix = pairwise_kernels(X_normalized, metric='rbf', gamma=gamma)
    df_affinity = pd.DataFrame(affinity_matrix)
    st.dataframe(df_affinity)
    st.markdown("""
    Matriks ini menunjukkan tingkat kedekatan atau kemiripan antar data berdasarkan fungsi kernel (di sini: **RBF kernel**).
    Nilai mendekati 1 artinya dua data sangat mirip.
    """)

    # Degree Matrix
    st.subheader("Degree Matrix")
    degree_matrix = np.diag(np.sum(affinity_matrix, axis=1))
    df_degree = pd.DataFrame(degree_matrix)
    st.dataframe(df_degree)
    st.markdown("""
    Matriks diagonal yang menyimpan jumlah total hubungan (kedekatan) setiap data terhadap semua data lainnya.
    Ini digunakan untuk menghitung Laplacian.
    """)

    # Laplacian Matrix
    st.subheader("Laplacian Matrix")
    laplacian_matrix = degree_matrix - affinity_matrix
    df_laplacian = pd.DataFrame(laplacian_matrix)
    st.dataframe(df_laplacian)
    st.markdown("""
    Laplacian Matrix diperoleh dari Degree Matrix dikurangi Affinity Matrix.
    Matriks ini penting dalam algoritma Spectral Clustering karena digunakan untuk menemukan struktur klaster alami dalam data.
    """)
    # Simpan ke session
    eigenvalues, eigenvectors = np.linalg.eigh(laplacian_matrix)
    st.session_state['X_normalized'] = X_normalized
    st.session_state['eigenvectors'] = eigenvectors[:, :2]
    st.session_state['df_processed'] = df

    # --- Hitung Eigenvalues & Eigenvectors ---
    from scipy.linalg import eigh
    from sklearn.metrics.pairwise import pairwise_kernels
    from scipy.sparse import csgraph

    # Gunakan Gaussian (RBF) kernel untuk affinity matrix
    gamma = 1.0
    affinity_matrix_gaussian = pairwise_kernels(X_normalized, metric='rbf', gamma=gamma)

    # Laplacian normed
    laplacian = csgraph.laplacian(affinity_matrix_gaussian, normed=True)

    # Eigen decomposition
    eigenvalues, eigenvectors = eigh(laplacian)

    # Simpan ke session
    st.session_state['X_normalized'] = X_normalized
    st.session_state['eigenvectors'] = eigenvectors[:, 1:3]  # ambil eigenvector ke-2 dan ke-3
    st.session_state['df_processed'] = df

    # --- Tampilkan Eigenvectors ---
    st.subheader("Eigenvectors (2D Embedding)")
    df_eigenvectors = pd.DataFrame(eigenvectors[:, 1:3], columns=["Eigenvector 2", "Eigenvector 3"])
    st.dataframe(df_eigenvectors)
    st.markdown("""
    Eigenvectors digunakan untuk memproyeksikan data ke ruang berdimensi lebih rendah
    sehingga memudahkan proses clustering.
    Pada Spectral Clustering, biasanya diambil beberapa eigenvector dengan eigenvalue terkecil.
    """)

# Menu Analisis
elif selected == "Analisis Klaster":
    # Pastikan data sudah diupload
    if 'df' not in st.session_state:
        if 'uploaded_file' in st.session_state:
            uploaded_file = st.session_state['uploaded_file']
            if uploaded_file.name.endswith('.csv'):
                df = pd.read_csv(uploaded_file, dtype=str)
            else:
                df = pd.read_excel(uploaded_file, engine='openpyxl', dtype=str)
            st.session_state['df'] = df
        else:
            st.warning("Data belum diupload. Silakan upload data dulu di menu 'Upload Data'.")
            st.stop()

    df = st.session_state['df']

    # Bersihkan dan persiapkan data
    df.columns = df.columns.str.strip()
    df.replace("-", np.nan, inplace=True)
    df.dropna(axis=1, thresh=0.1 * len(df), inplace=True)

    imputer = SimpleImputer(strategy="mean")
    for col in ['SD ke bawah', 'Diploma I/II/III/Akademi/Universitas']:
        if col in df.columns:
            df[col] = imputer.fit_transform(df[[col]])

    if "Wilayah" in df.columns:
        df["Wilayah_Original"] = df["Wilayah"]
        df["Wilayah_Encoded"] = LabelEncoder().fit_transform(df["Wilayah"])

    # Konversi kolom numerik
    numeric_columns = [
        "Jumlah angkatan Kerja", "Jumlah pengangguran terbuka", "TPT", "laki-laki", "perempuan",
        "SD ke bawah", "SMP", "SMA", "SMK", "Diploma I/II/III/Akademi/Universitas", "sk penetapan", "Aktif"
    ]
    for col in numeric_columns:
        if col in df.columns:
            df[col] = df[col].astype(str).str.replace(',', '', regex=True).replace('-', np.nan).astype(float)

    for col in ['Tahun', 'TPAK', 'TPT']:
        if col in df.columns:
            df[col] = pd.to_numeric(df[col], errors='coerce')

    # Pilih fitur untuk clustering
    features_for_cluster = [
        "TPT", "Jumlah pengangguran terbuka", "Aktif", "TPAK", "Jumlah angkatan Kerja",
        "laki-laki", "perempuan", "SD ke bawah", "SMP", "SMA", "SMK", "Diploma I/II/III/Akademi/Universitas"
    ]
    X = df[features_for_cluster].dropna()

    # Normalisasi data
    scaler = MinMaxScaler()
    X_normalized = scaler.fit_transform(X)

    # Membuat affinity matrix dengan kernel RBF
    gamma = 1.0
    affinity_matrix = pairwise_kernels(X_normalized, metric='rbf', gamma=gamma)

    # Hitung Laplacian matrix
    degree_matrix = np.diag(np.sum(affinity_matrix, axis=1))
    laplacian_matrix = degree_matrix - affinity_matrix

    # Eigen decomposition
    eigenvalues, eigenvectors = np.linalg.eigh(laplacian_matrix)

    # Ambil k eigenvector pertama
    k = 2
    eigenvector_subset = eigenvectors[:, :k]

    # Clustering spektral
    spectral = SpectralClustering(n_clusters=2, affinity='nearest_neighbors', random_state=42)
    cluster_labels = spectral.fit_predict(eigenvector_subset)

    # Simpan hasil cluster ke dataframe
    df.loc[X.index, 'Cluster'] = cluster_labels

    # Tetapkan cluster dominan per wilayah
    df['Cluster'] = df.groupby('Wilayah')['Cluster'].transform(lambda x: x.mode()[0])

    # Konversi ulang kolom numerik
    kolom_numerik = [
        'TPT', 'Jumlah pengangguran terbuka', 'Aktif', 'TPAK', 'Jumlah angkatan Kerja',
        'laki-laki', 'perempuan', 'SD ke bawah', 'SMP', 'SMA', 'SMK',
        'Diploma I/II/III/Akademi/Universitas'
    ]
    for kolom in kolom_numerik:
        df[kolom] = pd.to_numeric(df[kolom], errors='coerce')

    # Hitung rata-rata per klaster dan wilayah, simpan ke CSV
    average_cluster_data = df.groupby(['Wilayah', 'Cluster'])[kolom_numerik].mean().reset_index()
    average_cluster_data.to_csv('Rata-rata prakerja & Pengangguran perwilayah_klaster.csv', index=False)

    # Gabungkan koordinat wilayah dari file CSV eksternal
    coords = pd.read_csv("koordinat_wilayah.csv")
    df = df.merge(coords, on="Wilayah", how="left")

    # Bersihkan kolom yang tidak perlu
    kolom_drop = ['Wilayah_Original', 'Wilayah_Encoded', 'No', 'No.']
    df = df.drop(columns=[k for k in kolom_drop if k in df.columns])

    # Susun ulang kolom agar lebih terstruktur
    kolom_utama = ['Tahun', 'Wilayah', 'Cluster']
    kolom_lain = [col for col in df.columns if col not in kolom_utama]
    df = df[kolom_utama + kolom_lain]

    # Tampilkan data lengkap
    st.subheader("📋 Data Klasterisasi Lengkap")
    st.dataframe(df)

    # Form filter untuk tahun dan cluster
    with st.form("filter_form"):
        col1, col2 = st.columns(2)
        with col1:
            tahun_terpilih = st.selectbox("📅 Pilih Tahun", sorted(df['Tahun'].dropna().unique()))
        with col2:
            klaster_terpilih = st.multiselect("🔢 Pilih Klaster", sorted(df['Cluster'].dropna().unique()), default=sorted(df['Cluster'].dropna().unique()))
        filter_submit = st.form_submit_button("🔍 Terapkan Filter")

    if filter_submit:
        df_tahun = df[df['Tahun'] == tahun_terpilih]
        df_klaster = df_tahun[df_tahun['Cluster'].isin(klaster_terpilih)]

        st.subheader("📄 Data Klaster Terfilter")
        st.dataframe(df_klaster)

        st.markdown("""
               ### Hasil Klaster:
               Data wilayah dikelompokkan menjadi **2 klaster utama**:

               - **Cluster 0**:
                     - Cenderung memiliki TPT dan jumlah pengangguran yang lebih tinggi.
                     - Partisipasi Prakerja lebih besar → menunjukkan upaya intervensi terhadap pengangguran cukup aktif.
                     - Komposisi angkatan kerja lebih besar pada jenjang pendidikan menengah (SMA/SMK).

              - **Cluster 1**:
                    - Memiliki TPT yang lebih rendah dan jumlah pengangguran terbuka lebih sedikit**.
                    - Jumlah peserta aktif Prakerja relatif rendah, karena wilayah ini cenderung lebih stabil dari segi ketenagakerjaan.
                    - TPK lebih baik dan distribusi pendidikan cenderung lebih merata.

            """)
        # Konversi Cluster ke string agar cocok dengan color_discrete_map
        df_klaster = df_klaster.copy()
        df_klaster['Cluster'] = df_klaster['Cluster'].astype(int).astype(str)


        # Buat dictionary warna khusus
        # Mapping warna manual
        df_klaster['warna'] = df_klaster['Cluster'].map({
            '0': 'yellow',
            '1': 'blue'
        })
        # Plot manual pakai graph_objects
        fig = go.Figure()
        # Loop per klaster → satu trace saja per klaster
        for cluster_label in df_klaster['Cluster'].unique():
            df_sub = df_klaster[df_klaster['Cluster'] == cluster_label]
            fig.add_trace(go.Scattermapbox(
                lat=df_sub['lat'],
                lon=df_sub['lon'],
                mode='markers',
                marker=go.scattermapbox.Marker(
                    size=12,
                    color=df_sub['warna'].iloc[0]  # ambil warna konsisten
                ),
                name=f"Cluster {cluster_label}",
                text=df_sub.apply(lambda row: f"Wilayah: {row['Wilayah']}<br>Cluster: {row['Cluster']}", axis=1),
                hoverinfo='text'
        ))
        fig.update_layout(
            mapbox=dict(
                style="open-street-map",
                center=dict(lat=-7.0, lon=107.5),
                zoom=6
            ),
            showlegend=True,
            height=500
        )
        st.subheader("🗺️ Visualisasi Peta Klasterisasi Wilayah")
        st.plotly_chart(fig, use_container_width=True)
    else:
      st.warning("")

# Menu Rata-Rata
elif selected == "Rata-Rata Klaster":
    if 'df' not in st.session_state:
        st.warning("Data belum tersedia. Silakan lakukan upload dan analisis data terlebih dahulu.")
        st.stop()

    df = st.session_state['df']

    # Gunakan modus klaster per wilayah (klaster paling sering muncul)
    df['Cluster'] = df.groupby('Wilayah')['Cluster'].transform(lambda x: x.mode()[0])

    # Konversi kolom numerik ke tipe numerik
    kolom_numerik = [
        'TPT', 'Jumlah pengangguran terbuka', 'Aktif', 'TPAK', 'Jumlah angkatan Kerja',
        'laki-laki', 'perempuan', 'SD ke bawah', 'SMP', 'SMA', 'SMK',
        'Diploma I/II/III/Akademi/Universitas'
    ]
    for kolom in kolom_numerik:
        df[kolom] = pd.to_numeric(df[kolom], errors='coerce')

    # Hitung rata-rata indikator per wilayah dan klaster
    average_cluster_data = df.groupby(['Wilayah', 'Cluster'])[kolom_numerik].mean().reset_index()
    average_cluster_data.to_csv('Rata-rata prakerja & Pengangguran perwilayah_klaster.csv', index=False)

    # Hitung rata-rata per klaster (tanpa membedakan wilayah)
    rata_rata_klaster = df.groupby('Cluster')[kolom_numerik].mean().reset_index()

    # Tampilkan tabel hasil
    st.subheader("📊 Rata-rata Indikator per Tahun, Wilayah, dan Klaster")
    st.dataframe(average_cluster_data)

    st.subheader("📊 Rata-rata Indikator per Klaster")
    st.dataframe(rata_rata_klaster)

    # Penjelasan klaster 0
    st.markdown("""
        ### 🟡 Cluster 0 — Wilayah dengan Populasi Kerja Besar dan Pengangguran Tinggi

        - **TPT**: `8.95%`
        - **Jumlah Pengangguran Terbuka**: `137.209 orang`
        - **Peserta Aktif**: `37.581 orang`
        - **TPAK**: `65.77%`
        - **Jumlah Angkatan Kerja**: `1.513.606 orang`

        **Komposisi Pendidikan Pengangguran:**
        - SD ke bawah: `29.165`
        - SMP: `26.386`
        - SMA: `33.946`
        - SMK: `35.560`
        - Diploma/Universitas: `18.866`

        **Karakteristik:**
        Wilayah dengan jumlah pengangguran tinggi, didominasi oleh lulusan SMK dan SMA.
        Kemungkinan besar merupakan wilayah perkotaan dengan tingkat kompetisi kerja tinggi.
    """)

    # Penjelasan klaster 1
    st.markdown("""
        ### 🔵 Cluster 1 — Wilayah dengan Skala Kecil dan Pengangguran Lebih Rendah

        - **TPT**: `7.65%`
        - **Jumlah Pengangguran Terbuka**: `41.636 orang`
        - **Peserta Aktif**: `12.236 orang`
        - **TPAK**: `66.63%`
        - **Jumlah Angkatan Kerja**: `558.028 orang`

        **Komposisi Pendidikan Pengangguran:**
        - SD ke bawah: `16.158`
        - SMP: `13.880`
        - SMA: `12.981`
        - SMK: `13.955`
        - Diploma/Universitas: `12.589`

        **Karakteristik:**
        Wilayah dengan jumlah tenaga kerja dan penganggur yang lebih kecil.
        Umumnya merupakan wilayah rural atau kota kecil dengan tekanan kerja lebih rendah.
    """)

    # ======= VISUALISASI PETA RATA-RATA PER KLASTER =======

    try:
        coords = pd.read_csv("koordinat_wilayah.csv")

        # Gabungkan koordinat dengan data rata-rata
        average_cluster_map = average_cluster_data.merge(coords, on='Wilayah', how='left')
        average_cluster_map = average_cluster_map.dropna(subset=['lat', 'lon'])

        # Konversi cluster ke string & buat kolom warna
        average_cluster_map['Cluster'] = average_cluster_map['Cluster'].astype(int).astype(str)
        average_cluster_map['warna'] = average_cluster_map['Cluster'].map({
            '0': 'yellow',
            '1': 'blue'
        })

        # Buat peta
        fig = go.Figure()

        for cluster_label in average_cluster_map['Cluster'].unique():
            df_sub = average_cluster_map[average_cluster_map['Cluster'] == cluster_label]

            if not df_sub.empty and pd.notna(df_sub['warna'].iloc[0]):
                fig.add_trace(go.Scattermapbox(
                    lat=df_sub['lat'],
                    lon=df_sub['lon'],
                    mode='markers',
                    marker=go.scattermapbox.Marker(
                        size=14,
                        color=df_sub['warna'].iloc[0]
                    ),
                    name=f"Cluster {cluster_label}",
                    text=df_sub.apply(lambda row: f"Wilayah: {row['Wilayah']}<br>Cluster: {row['Cluster']}", axis=1),
                    hoverinfo='text'
                ))

        # Layout peta
        fig.update_layout(
            mapbox=dict(
                style="open-street-map",
                center=dict(
                    lat=average_cluster_map['lat'].mean(),
                    lon=average_cluster_map['lon'].mean()
                ),
                zoom=6
            ),
            showlegend=True,
            height=500
        )

        st.subheader("🗺️ Peta Rata-rata Indikator per Klaster")
        st.plotly_chart(fig, use_container_width=True)

    except Exception as e:
        st.warning(f"❗Tidak ada data koordinat rata-rata indikator yang tersedia untuk divisualisasikan.\n\nDetail: {e}")

# Menu Analisis Pengaruh Prakerja terhadap Pengangguran
elif selected == "Analisis Pengaruh Prakerja terhadap Pengangguran":
    if 'df' not in st.session_state:
        st.warning("Data belum diupload. Silakan upload data terlebih dahulu.")
        st.stop()

    df = st.session_state['df']
    df.columns = df.columns.str.strip()

    # --- Bagian 1: Korelasi Prakerja dengan Pengangguran dan Pendidikan ---
    st.header("Korelasi Prakerja terhadap Pengangguran dan Pendidikan")

    kolom_utama = ['Aktif', 'TPT']
    kolom_pendidikan = ['Aktif', 'SD ke bawah', 'SMP', 'SMA', 'SMK', 'Diploma I/II/III/Akademi/Universitas']

    if not all(kol in df.columns for kol in kolom_utama):
        st.error("Kolom 'Aktif' dan/atau 'TPT' tidak tersedia di dataset.")
    elif not all(kol in df.columns for kol in kolom_pendidikan):
        st.error("Beberapa kolom pendidikan tidak tersedia di dataset.")
    else:
        # Korelasi Aktif dan TPT
        korelasi_aktif_tpt = df[kolom_utama].corr()
        st.subheader("📌 Korelasi antara 'Aktif' dan 'TPT'")
        st.dataframe(korelasi_aktif_tpt)
        st.markdown("""
        **Interpretasi Hasil Korelasi (Aktif dan TPT)**

        - **Nilai korelasi:** 0,35 (hubungan sedang dan positif)
        - **Makna:** Partisipasi aktif dalam Program Kartu Prakerja berkaitan dengan tingkat pengangguran, namun tidak kuat.
        - **Kesimpulan:**
        - Program Prakerja belum berpengaruh signifikan terhadap penurunan pengangguran.
        - Faktor lain seperti pasar kerja dan keterampilan tetap berperan penting.
        """)

        # Korelasi Aktif dan Pendidikan
        korelasi_aktif_pendidikan = df[kolom_pendidikan].corr()
        st.subheader("📌 Korelasi antara 'Aktif' dan Jenjang Pendidikan")
        st.dataframe(korelasi_aktif_pendidikan)
        st.markdown("""
        **Penjelasan**

        - **Peserta Prakerja (Aktif)** paling banyak berasal dari lulusan **SMA (korelasi 0.64)** dan **SMK (0.63)**.
        - Peserta dari lulusan **SD ke bawah (0.41)** dan **SMP (0.39)** ada juga, tapi jumlahnya lebih sedikit.
        - Peserta dari lulusan **Diploma/Universitas (0.05)** sangat sedikit atau hampir tidak ada.

        **Kesimpulan:** Program Prakerja lebih diminati oleh lulusan SMA dan SMK, sementara lulusan perguruan tinggi relatif sedikit yang ikut.
        """)
    st.header("Pengaruh program Kartu Prakerja")
    required_cols = {'Tahun', 'Aktif', 'TPT'}
    if not required_cols.issubset(df.columns):
        st.error("Data harus memiliki kolom 'Tahun', 'Aktif', dan 'TPT'.")
        st.stop()

    # Bagi berdasarkan partisipasi tinggi vs rendah
    high_participation = df[df['Aktif'] > df['Aktif'].median()]
    low_participation = df[df['Aktif'] <= df['Aktif'].median()]

    # Tahun awal program Prakerja (2020)
    high_participation_2020 = high_participation[high_participation['Tahun'] == 2020]['TPT']
    high_participation_after = high_participation[high_participation['Tahun'] >= 2021]['TPT']

    low_participation_2020 = low_participation[low_participation['Tahun'] == 2020]['TPT']
    low_participation_after = low_participation[low_participation['Tahun'] >= 2021]['TPT']

    # Hitung rata-rata TPT
    avg_tpt_high_2020 = high_participation_2020.mean()
    avg_tpt_high_after = high_participation_after.mean()

    avg_tpt_low_2020 = low_participation_2020.mean()
    avg_tpt_low_after = low_participation_after.mean()

    # Tampilkan hasil
    st.markdown(f"**Wilayah dengan partisipasi tinggi Prakerja:**")
    st.markdown(f"- TPT tahun 2020 (awal program): **{avg_tpt_high_2020:.2f}%**")
    st.markdown(f"- TPT tahun 2021–2024: **{avg_tpt_high_after:.2f}%**")

    st.markdown(f"**Wilayah dengan partisipasi rendah Prakerja:**")
    st.markdown(f"- TPT tahun 2020 (awal program): **{avg_tpt_low_2020:.2f}%**")
    st.markdown(f"- TPT tahun 2021–2024: **{avg_tpt_low_after:.2f}%**")

    st.markdown("""
    - TPT menurun di wilayah dengan **partisipasi tinggi** maupun **rendah** dalam program Prakerja.
    - Penurunan lebih besar terjadi di wilayah dengan **partisipasi tinggi**.
    - Namun, karena penurunan juga terjadi di wilayah partisipasi rendah, **Prakerja bukan satu-satunya faktor**.
    - Faktor lain seperti **kondisi ekonomi dan kebijakan pemerintah** juga berperan.
    """)

    # Visualisasi perbandingan
    st.subheader("📈 Visualisasi Perbandingan")

    chart_data = pd.DataFrame({
        'Kategori': ['Tinggi 2020', 'Tinggi 2021–2024', 'Rendah 2020', 'Rendah 2021–2024'],
        'TPT': [avg_tpt_high_2020, avg_tpt_high_after, avg_tpt_low_2020, avg_tpt_low_after]
    })

    fig, ax = plt.subplots()
    ax.bar(chart_data['Kategori'], chart_data['TPT'], color=['blue', 'blue', 'orange', 'orange'])
    ax.set_ylabel("Tingkat Pengangguran Terbuka (%)")
    ax.set_title("Perbandingan Rata-rata TPT Berdasarkan Partisipasi Prakerja")
    st.pyplot(fig)

# Menu Input Data Manual
elif selected == "Input Data Manual":

    # Menu Input Data Manual
    st.header("📝 Input Data Manual untuk Cluster")

    if 'df' not in st.session_state:
        st.warning("Data belum diupload. Silakan upload data dulu di menu Upload Data.")
    else:
        df = st.session_state['df']

        features_for_cluster = [
            "TPT", "Jumlah pengangguran terbuka", "Aktif", "TPAK", "Jumlah angkatan Kerja",
            "laki-laki", "perempuan", "SD ke bawah", "SMP", "SMA", "SMK", "Diploma I/II/III/Akademi/Universitas"
        ]

        missing_cols = [col for col in features_for_cluster if col not in df.columns]
        if missing_cols:
            st.error(f"Kolom berikut tidak ditemukan dalam data: {missing_cols}")
            st.write("Kolom-kolom yang ada di data:")
            st.write(df.columns.tolist())
            st.stop()

        X = df[features_for_cluster].astype(float)
        scaler = StandardScaler()
        X_normalized = scaler.fit_transform(X)

        gamma = 0.1
        k = 2

        if 'Wilayah' in df.columns:
            wilayah_list = df['Wilayah'].dropna().unique().tolist()
            if not wilayah_list:
                wilayah_list = ["N/A"]
        else:
            wilayah_list = ["N/A"]

        with st.form("form_input_manual"):
            input_data = {}
            cols = st.columns(3)

            for i, feature in enumerate(features_for_cluster):
                with cols[i % 3]:
                    input_data[feature] = st.number_input(f"{feature}", value=0.0, step=1.0)

            input_tahun = st.number_input("Tahun", min_value=2000, max_value=2100, value=2023, step=1)
            input_wilayah = st.selectbox("Wilayah", options=wilayah_list)

            submitted = st.form_submit_button("🔍 Klaster")

        if submitted:
            try:
                input_df = pd.DataFrame([input_data])

                # Validasi apakah ada NaN
                if input_df.isnull().values.any():
                    st.error("❌ Input Anda mengandung nilai kosong (NaN). Harap isi semua kolom dengan benar.")
                    st.stop()

                input_normalized = scaler.transform(input_df)
                combined_normalized = np.vstack([X_normalized, input_normalized])

                combined_affinity = pairwise_kernels(combined_normalized, metric='rbf', gamma=gamma)
                combined_degree = np.diag(np.sum(combined_affinity, axis=1))
                combined_laplacian = combined_degree - combined_affinity

                _, combined_eigenvectors = np.linalg.eigh(combined_laplacian)
                combined_subset = combined_eigenvectors[:, :k]

                spectral = SpectralClustering(n_clusters=k, affinity='nearest_neighbors', random_state=42)
                combined_labels = spectral.fit_predict(combined_subset)

                predicted_cluster = combined_labels[-1]

                st.success(f"✅ Data yang Anda input masuk ke **Cluster {predicted_cluster}**.")
                st.markdown("**Detail Data yang Dimasukkan:**")
                st.write(input_df)
                st.markdown(f"📅 Tahun: **{input_tahun}**  | 🌍 Wilayah: **{input_wilayah}**")

                # 🔍 Korelasi Antar Fitur
                st.subheader("🔍 Korelasi Antar Fitur")
                selected_features = [
                    'Aktif', 'TPT',
                    'SD ke bawah', 'SMP', 'SMA', 'SMK',
                    'Diploma I/II/III/Akademi/Universitas'
                ]

                 # Cek apakah semua input bernilai nol
                if all(value == 0.0 for value in input_data.values()):
                    st.warning("📭 Anda belum mengisi data apapun. Korelasi tidak dapat dihitung dari input kosong.")
                else:
                    # Simulasikan 10 baris dari input user dengan variasi kecil
                    simulated_inputs = pd.DataFrame([input_data] * 10)
                    seed_value = int(hashlib.md5(str(input_data).encode()).hexdigest(), 16) % (2**32)
                    np.random.seed(seed_value)
                    noise = np.random.normal(0, 0.01, simulated_inputs[selected_features].shape)
                    simulated_inputs[selected_features] += noise

                    corr_matrix = simulated_inputs[selected_features].corr()
                    st.dataframe(
                        corr_matrix.style.background_gradient(cmap='coolwarm').format("{:.3f}")
                    )

                    st.markdown("""
                    Penjelasan Korelasi:
                    - Nilai **mendekati 1** atau **-1** artinya hubungan **kuat**.
                    - Nilai **mendekati 0** artinya hubungan **lemah**

                    """)

            except Exception as e:
                st.error(f"❌ Terjadi error saat memproses data: {e}")


Writing app.py


In [ ]:
!ngrok config add-authtoken 2w7tzu4F7z7FY97gMXIlVIrPywM_4ofdSWDtbLssEUtPVJ5pc

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:
from pyngrok import ngrok

# Buka port 8501 untuk Streamlit
public_url = ngrok.connect(8501)  # Tanpa 'port='
print("URL publiknya:", public_url)


# Jalanin Streamlit
!streamlit run app.py &


URL publiknya: NgrokTunnel: "https://d906d2c8a79a.ngrok-free.app" -> "http://localhost:8501"



  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://35.229.37.9:8501

